In [1]:
print("Hello ")

Hello 


In [2]:
import pandas as pd


In [2]:

# Data Manipulation
import pandas as pd
import numpy as np

# RegEx and String Manipulation
import re
import string

# Language Detection
from nltk.classify.textcat import TextCat

# Multiprocessing
import dask.dataframe as dd
import multiprocessing

# BERT-Embeddings
from keybert import KeyBERT

# TF-IDF Vectorization
from sklearn.feature_extraction.text import TfidfVectorizer

# Plotting Heatmap of TF-IDF vectors 
import matplotlib.pyplot as plt
import altair as alt
alt.renderers.enable('mimetype')

# Cosine Similarity
from sklearn.metrics.pairwise import cosine_similarity

C:\Users\public\anaconda3\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [3]:
# Minimum number of words that should be present in a description (value starting from 1)
min_description_word_count = 3

In [4]:
books_data = pd.read_csv("C://Users/personal/Downloads/goodreadbooks/book3000k-4000k.csv", usecols=['Id', 'Name', 'Authors', 'ISBN', 'PublishYear', 'Publisher', 'Language', 'Description'])
display(books_data.shape)
books_data.head(5)

(256595, 8)

,Id,Name,Authors,ISBN,PublishYear,Publisher,Language,Description
0,3000006,Simple Sermons on Great Christian Doctrines,W. Herschel Ford,0801091241,2001,Baker Books,NaN,A collection of simple sermons to help busy pa...
1,3000007,The Spinoza of Market Street,Isaac Bashevis Singer,0374502560,1979,"Farrar, Straus and Giroux",NaN,NaN
2,3000008,The Golem,Isaac Bashevis Singer,0374427461,1996,Farrar Straus Giroux,NaN,A clay giant miraculously brought to life by a...
3,3000009,Ujamaa Villages in Tanzania: Analysis of a Soc...,Michaela Von Freyhold,0853455120,1981,Monthly Review Press,NaN,NaN
4,3000011,Introducing Special Educational Needs: A Guide...,Philip Gardner,1853467332,2001,David Fulton Publishers,NaN,Pressure of time means that the complex topic ...


#  Text Cleaning 

In [7]:
books_data.dropna(subset=["Description"], inplace=True)

In [5]:
max(books_data.Id)

3724836

In [6]:
list(books_data.Description[books_data.Id == 3724836]) #Description with url and html tag


['Mennonites still comprise one of the most markedly rural ethno-religious groups in North America, but are urbanizing faster than other groups. This study illustrates how they have survived, how they are changing, and how they have dealt with internal and external conflict in the process.']

In [8]:
url_pattern = re.compile(r'https?://\S+|www\.\S+')
def remove_url(text):   
    return re.sub(url_pattern, r'', text)

html_pattern = re.compile('<[^>]*>')
def clean_html_tags(text):
    return re.sub(html_pattern, r'', text)

punctuations = string.punctuation
def remove_punctuations(text):
    return text.translate(str.maketrans('', '', punctuations))

books_data.Description = books_data.Description.apply(remove_url)
books_data.Description = books_data.Description.apply(clean_html_tags)
books_data.Description = books_data.Description.apply(remove_punctuations)

# Result
list(books_data.Description[books_data.Id == 3724836])

TypeError: expected string or bytes-like object

#### Convert Letter Case to Lower and Clip Extra Spaces

In [9]:
books_data[["Publisher"]] = books_data[["Publisher"]].fillna("unknown")
books_data[["Name", "Authors", "Publisher", "Description"]] = pd.concat([books_data[col].astype(str).str.lower().str.strip() 
                                                                             for col in ["Name", "Authors", "Publisher", "Description"]], 
                                                                            axis=1)

In [10]:
books_data.head(10)

,Id,Name,Authors,ISBN,PublishYear,Publisher,Language,Description
0,3000006,simple sermons on great christian doctrines,w. herschel ford,0801091241,2001,baker books,NaN,a collection of simple sermons to help busy pa...
1,3000007,the spinoza of market street,isaac bashevis singer,0374502560,1979,"farrar, straus and giroux",NaN,nan
2,3000008,the golem,isaac bashevis singer,0374427461,1996,farrar straus giroux,NaN,a clay giant miraculously brought to life by a...
3,3000009,ujamaa villages in tanzania: analysis of a soc...,michaela von freyhold,0853455120,1981,monthly review press,NaN,nan
4,3000011,introducing special educational needs: a guide...,philip gardner,1853467332,2001,david fulton publishers,NaN,pressure of time means that the complex topic ...
5,3000014,my lady of cleves,margaret campbell barnes,1402214316,2008,sourcebooks landmark,eng,written by world-reknowned historical novelist...
6,3000015,women and religious writing in early modern en...,erica longfellow,0521837588,2004,cambridge university press,NaN,challenging critical assumptions about the rol...
7,3000023,package printing,nelson eldred,0961630256,1998,crc press,NaN,nan
8,3000024,meat refrigeration,s.j. james,0849315387,2002,crc press,NaN,the chilling and freezing of meat remains an e...
9,3000025,kinematic geometry of surface machining,stephen p. radzevich,1420063405,2007,crc press,NaN,the principle of occam's razor loosely transla...


## Remove Book Descriptions With Shorter Length.

In [11]:
# Find description word count
books_data["length"] = [len(d.split()) for d in books_data['Description'].tolist()]

print(set(books_data.Description[books_data.length.isin(range(0,4))]))


{'two volumes.', '.', '八丁標の外に出てはいけない―悪鬼と業魔から町を守るために、大人たちが作った忌まわしい伝説。いま伝説が、「実体」となって町に迫る。新しい秩序とは、おびただしい流血でしか生まれないのか。少女は、決死の冒険に身を投じる。', '<i>east asian books</i>', 'country inspired rooms', '根気も集中力もなく、部屋は乱雑、でもそれがおれのライフスタイルさ……などとうそぶく人生の敗者エディが手にした「脳を活性化する薬」。服（の）めば気合充溢、部屋は片づき、企画力も抜群、株では大儲け、ついには実業界の大物の片腕に。でも、そんなうまい話があると思います？\u3000身におぼえがありすぎて、ちっとも洒落にならない悪夢と破滅の物語。英国推理作家協会新人賞候補、勝ち組と負け組が露骨に表われるニューヨークのビジネス界の世知辛さも見事に描かれています。', '私は他の女の子たちよりも早く老けるだろう。チャイルドモデルから芸能界へ―幼い頃からtvの中で生きてきた美しくすこやかな少女・夕子。ある出来事をきっかけに、彼女はブレイクするが…少女の心とからだに流れる18年の時間を描く。芥川賞受賞第一作。', '<br /><br />', '仇敵を討つ！\u3000修業に励むナルトに届くアスマの訃報。悲しみに暮れる間もなく、第十班はカカシと共に再び木ノ葉へ潜入する“暁”と対峙する。シカマルの作戦で優位に戦いを進める中、角都の驚愕の能力が明らかに！！', 'no synopsis available.', '...', '《呼蘭河傳》是民國才女蕭紅在香港創作的一部長篇小説，也是她的巔峰之作。小說以作者的家鄉和童年生活為原型，描繪出了作者記憶中的家鄉呼蘭——上世紀二、三十年代一個東北小鎮的單調與美麗、人民的善良與愚昧。全書七章：一、二章寫小城風情，三、四章談家中親疏人物，五、六、七章摹繪獨立旁枝人物。七章可各自獨立又渾然一體。', 'western', 'zwei leidenschaftliche liebesromane.', 'no', '親友のバニラに後れをとりながらも、ショコラは魔界の女王（クイーン）を目指し、人間界で修業中。一方、妖しい魅力の美少年・ピエールが、何の企みか、ショコラに近づいてきて……。大

In [12]:
# Replace empty strings of description with NaN
books_data.Description = books_data.Description.replace(r'^\s*$', np.nan, regex=True)

books_data[books_data.length.isin(range(1,min_description_word_count+1))][["Id", "Name", "Description", "length"]]\
.sort_values(by=["length"], ascending=True).head(5)

,Id,Name,Description,length
1,3000007,the spinoza of market street,nan,1
188574,3532943,world history: a dictionary of important peopl...,nan,1
188575,3532944,tax simplification bills: hearings before the ...,nan,1
188576,3532945,capital punishment: the inevitability of capri...,nan,1
188577,3532947,pop photographica: photography's objects in ev...,nan,1


In [13]:
books_data.dropna(subset=["Description"], inplace=True)

# Drop records with very short description
books_data.drop(books_data.index[books_data.length.isin(range(0,min_description_word_count+1))], inplace = True)
del books_data["length"]

In [14]:
# Convert unknown to NaN
books_data["Publisher"] = books_data.Publisher.replace('unknown',np.nan)
books_data = books_data.sort_values(by="Publisher", na_position='last')\
.drop_duplicates(subset=["Name", "Authors", "Description"], keep='first')

### Extract and Remove Book Series Information from the Book Name

In [15]:
series_pattern =  "(?:[;]\s*|\(\s*)([^\(;]*\s*#\s*\d+(?:\.?\d+|\\&\d+|-?\d*))"
def get_book_series_info(text):
    series_info = re.findall(series_pattern, text)
    if series_info:
        series_info = " ".join([i.replace(" ", "_") for i in series_info])
        return series_info
    else:
        return np.nan
    
books_data['BookSeriesInfo'] = books_data.Name.apply(get_book_series_info)

In [16]:
series_remove_pattern = re.compile("(?:[\(]\s*[^\(;]*\s*#\s*\d+(?:\.?\d+|\\&\d+|-?\d*)(?:;|\))|\s*[^\(;]*\s*#\s*\d+(?:\.?\d+|\\&\d+|-?\d*)\))")       
def remove_series_info(text):
    return re.sub(series_remove_pattern, r'', text)

books_data["Title"]= books_data["Name"].str.replace(series_remove_pattern, r'').str.strip()


###  Impute Missing Language Information Using the Language of the Book Name

In [17]:
tc = TextCat()

def detect_language(text):
    text = " ".join(text.split()[:5])
    if text.isnumeric():
        return 'eng'
    else:
        return tc.guess_language(text).strip()

In [ ]:
"""
 Takes longer time to process thousands records hence results are presaved in preprocessed.csv
 """
ddf = dd.from_pandas(books_data, npartitions=4*multiprocessing.cpu_count()) 
books_data["Language"] = ddf.map_partitions(lambda df: df.apply(lambda x: detect_language(x['Name']) if pd.isna(x['Language']) else x['Language'], axis=1)).compute() 
books_data.isna().sum()

In [ ]:
temp_preview = books_data.head(5).copy()
ddf = dd.from_pandas(temp_preview, npartitions=4*multiprocessing.cpu_count()) 
temp_preview["Language"] = ddf.map_partitions(lambda df: df.apply(lambda x: detect_language(x['Name']) if pd.isna(x['Language']) else x['Language'], axis=1)).compute() 
temp_preview

## Remove Double Quotes from Publisher Name.

In [ ]:
books_data["Publisher"] = books_data["Publisher"].str.replace('"','')

## Transform Book and Author Names into Single Token

In [ ]:
books_data["Authors"] = books_data["Authors"].str.strip().str.replace(' ','_')
books_data["Publisher"] = books_data["Publisher"].str.strip().str.replace(' ','_')
books_data.head(5)

###  Merge All the Textual Summary into a Single Summary Column.

In [ ]:
books_data["bow"] = eda_data[["BookSeriesInfo", 'Authors', 'Publisher', 'Language']].fillna('').agg(' '.join, axis=1)

In [ ]:
books_data.bow.iloc[8375]

In [ ]:
# Save cleaned data
books_data.to_csv("C://Users/personal/Downloads/goodreadbooks/preprocessed.csv", sep=",", index=False)

#  Feature Engineering

In [ ]:
# Fetch preprocessed cleaned data
fe_data = pd.read_csv("C://Users/personal/Downloads/goodreadbooks/preprocessed.csv", usecols=["Id", "Name", "Language", "Description", "bow"])
fe_data.head()

## Keyword Extraction Using KeyBERT
#### Use keyBERT to extract relevant keywords from the Description. It is developed and maintained by Maarten Grootendorst. As stated in its document KeyBERT uses BERT-embeddings and cosine similarity to find the sub-phrases in a document that are the most similar to the document itself.

In [ ]:
kw_model = KeyBERT()

def get_keywords(text):
    keywords = kw_model.extract_keywords(text, keyphrase_ngram_range=(1, 1), stop_words="english")
    keywords = " ".join([k[0] for k in keywords])
    return keywords

fe_data["keywords"] = fe_data.Description.apply(get_keywords)

In [ ]:
fe_data.keywords.head()

In [ ]:
fe_data["keywords"] = fe_data[['bow', 'keywords']].fillna('').agg(' '.join, axis=1)
fe_data.drop(['bow', 'Description'], axis = 1, inplace=True)

###  Remove duplicated Book Names

In [ ]:
fe_data = fe_data.drop_duplicates(subset=["Name"], keep='first')

In [ ]:
# Store results
fe_data.to_csv("C://Users/personal/Downloads/goodreadbooks/keywords.csv", sep=",", index=False)

# Text Representation

In [ ]:
# Fetch keywords data
model_data = pd.read_csv("C://Users/personal/Downloads/goodreadbooks/keywords.csv")
model_data.head()

# Vectorize the Keywords Summary Using TF-IDF

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(analyzer = 'word',
                        min_df=3,
                        max_df = 0.6,
                        stop_words="english",
                        encoding = 'utf-8', 
                        token_pattern=r"(?u)\S\S+")
tfidf_encoding = tfidf.fit_transform(model_data["keywords"])

In [ ]:
# Preview first 100 words in the vocabulary
print(tfidf.get_feature_names_out()[1:100])

### Visualize the TF-IDF word embeddings

In [ ]:
tfidf_df = pd.DataFrame(tfidf_encoding.toarray(), index=model_data["Name"], columns=tfidf.get_feature_names_out())

# Find top 50 books with maximum tf-idf total score
tfidf_df["total"]= tfidf_df.sum(axis=1)
tfidf_df = tfidf_df.sort_values("total", ascending=False)
del tfidf_df["total"]

In [ ]:
# Leave first few words containing years and select top 50 books
tfidf_df_preview = tfidf_df.iloc[100:150,25:].copy()
tfidf_df_preview = tfidf_df_preview.stack().reset_index()
tfidf_df_preview = tfidf_df_preview.rename(columns={0:'tfidf', 'Name': 'book','level_1': 'term'})
tfidf_df_preview = tfidf_df_preview.sort_values(by=['book','tfidf'], ascending=[True,False]).groupby(['book']).head(10)
display(tfidf_df_preview)

In [ ]:
def process_word_matrix(word_vec):
    # Remove underscores in terms
    word_vec.term = word_vec.term.str.replace('_',' ')

    # Remove terms with zero tfidf score
    word_vec = word_vec[word_vec.tfidf > 0]
    
    return word_vec

tfidf_vec = process_word_matrix(tfidf_df_preview.copy())    
tfidf_vec.iloc[0:5]

In [ ]:
import altair as alt
grid = alt.Chart(tfidf_vec).encode(
    x = 'rank:O',
    y = 'book:N'
).transform_window(
    rank = "dense_rank()",
    sort = [alt.SortField("tfidf", order="descending")],
    groupby = ["book"],
)
heatmap = grid.mark_rect(size=5).encode(
    alt.Color('tfidf:Q', scale=alt.Scale(scheme='redpurple'))
)
text = grid.mark_text(align='center', baseline='middle', lineBreak='').encode(
    text = 'term:N',
    color = alt.condition(alt.datum.tfidf >= 0.23, alt.value('white'), alt.value('black'))
)
(heatmap+text).properties(width = 800)

# Cosine Similarity Between Books Vector Representation

######

Once the numerical vector representation of the data is created for each book, it becomes possible to apply all the techniques applicable in a geometric space. It is possible to find similarity between two vectors (and thereby between books represented as vectors). 

In [ ]:
book_cosine_sim = cosine_similarity(tfidf_encoding, tfidf_encoding)

In [ ]:

# Preview Similarity Matrix
book_cosine_sim

In [ ]:
# Vidualize similarity between books
plt.figure(figsize=(6, 6), dpi=80)
plt.spy(book_cosine_sim, precision = 0.1, markersize = 0.04)
plt.tight_layout()
plt.show()

# Recommendation
Purpose is: given a book name, find top n similar books based on cosine similarity score. In real use cases, the input book could be the book a user has read, has rated highly or have added to the read later list. Books are recommended utilising the following information through keywords:

In [ ]:
books = pd.Series(model_data['Name'])

def recommend_books_similar_to(book_name, n=5, cosine_sim_mat=book_cosine_sim):
    # get index of the imput book
    input_idx = books[books == book_name].index[0]   
    # Find top n similar books with decreasing order of similarity score
    top_n_books_idx = list(pd.Series(cosine_sim_mat[input_idx]).sort_values(ascending = False).iloc[1:n+1].index)
    # [1:6] to exclude 0 (index 0 is the input movie itself)
    
    books_list = list(books)
    recommended_books = [books[i] for i in top_n_books_idx]
        
    return recommended_books

# 1. Series Information
Suggest other books from the similar series.
In the example, series information images of america is used to recommend books relating to different parts of America from the same series (new jersey). 

In [ ]:
# Recommendations with series information
print("\033[1m{}\033[0m".format("Recommendation (Series Information) based on the read: The Eastland Disaster (Images of America: Illinois)"))
display(recommend_books_similar_to("the eastland disaster (images of america: illinois)", 5))

# 2. Other Books In Numbered Series
Suggest other books in the sequence from the same series.
In the example below, other books in the series sequence of Antique Lover are recommended.

In [ ]:
# Recommendations with series information numbered
print("\n\033[1m{}\033[0m".format("Recommendation (Numbered Series) based on the read: The Majolica Murders (Antique Lover, #5)"))
display(recommend_books_similar_to("the majolica murders (antique lover, #5)", 5))

# 3.Theme
Utilise keywords depicting semantic meaning of the theme of the book to suggest books from similar themes.
In the example below, all the books about developing programming skills irrespective of programming language are recommended.

In [ ]:
print("\n\033[1m{}\033[0m".format("Recommendation (Theme: Programming) based on the read: The Practice of Programming (Addison-Wesley Professional Computing Series)"))
display(recommend_books_similar_to('the practice of programming (addison-wesley professional computing series)', 5))

# 4. Author
Suggest other works of the same Author.

In [ ]:
print("\n\033[1m{}\033[0m".format("Recommendation (Author: Dean Koontz) based on the read: Cold Fire"))
display(recommend_books_similar_to("cold fire",5))